Renan's notebook

In [3]:
# Uncomment the following if converting from AVIF to JPEG

'''
from PIL import Image
import pillow_avif  # noqa: F401, needed to register AVIF support

img = Image.open("/Users/renan/code/reneiji/non_verbal_comm/serious-young-african-man.avif")
img = img.convert("RGB")
img.save("serious-young-african-man.jpg", "JPEG")
'''

'\nfrom PIL import Image\nimport pillow_avif  # noqa: F401, needed to register AVIF support\n\nimg = Image.open("/Users/renan/code/reneiji/non_verbal_comm/serious-young-african-man.avif")\nimg = img.convert("RGB")\nimg.save("serious-young-african-man.jpg", "JPEG")\n'

In [1]:
from deepface import DeepFace
import cv2
import mediapipe as mp
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

from datasets import load_dataset

In [ ]:
# Analyze an image (local path or URL)
result = DeepFace.analyze(img_path = "/Users/renan/code/reneiji/non_verbal_comm/notebooks/RA_note/threatening_smile.jpeg", actions = ['emotion'])

print(result[0]['emotion'])  # prints a dictionary with emotion scores

{'angry': 0.03257695255349397, 'disgust': 1.1144623255532201e-11, 'fear': 0.002219444749328375, 'happy': 99.84859825985453, 'sad': 0.0025796688186143445, 'surprise': 0.011756884540774653, 'neutral': 0.10226941612503933}


## This is the results for cropping mouth and eyes

In [7]:
import cv2
import mediapipe as mp
from deepface import DeepFace

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

img_path = "/Users/renan/code/reneiji/non_verbal_comm/notebooks/serious-young-african-man.jpg"
img = cv2.imread(img_path)
if img is None:
    print(f"⚠ Failed to load image. Check path: {img_path}")
    exit()
h, w, _ = img.shape
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = face_mesh.process(rgb_img)

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:

        # Define landmark indices for eyes + mouth regions
        eye_indices = [33, 133, 362, 263]  # corners of both eyes
        mouth_indices = [61, 291, 0, 17]   # mouth corners, chin, nose

        def expand_bbox(indices, expand_x_ratio=0.5, expand_y_ratio=0.6):
            xs = [int(face_landmarks.landmark[i].x * w) for i in indices]
            ys = [int(face_landmarks.landmark[i].y * h) for i in indices]
            x_min, x_max = min(xs), max(xs)
            y_min, y_max = min(ys), max(ys)
            box_w = x_max - x_min
            box_h = y_max - y_min
            # Expand the box
            x_min = max(0, int(x_min - box_w * expand_x_ratio))
            x_max = min(w, int(x_max + box_w * expand_x_ratio))
            y_min = max(0, int(y_min - box_h * expand_y_ratio))
            y_max = min(h, int(y_max + box_h * expand_y_ratio))
            return x_min, y_min, x_max, y_max

        # Get expanded regions
        eye_x1, eye_y1, eye_x2, eye_y2 = expand_bbox(eye_indices,expand_x_ratio=0.5, expand_y_ratio=5)
        mouth_x1, mouth_y1, mouth_x2, mouth_y2 = expand_bbox(mouth_indices, expand_x_ratio=0.8, expand_y_ratio=2)

        eye_crop = img[eye_y1:eye_y2, eye_x1:eye_x2]
        mouth_crop = img[mouth_y1:mouth_y2, mouth_x1:mouth_x2]

        # Optional: save crops for inspection
        cv2.imwrite("eye_region.jpg", eye_crop)
        cv2.imwrite("mouth_region.jpg", mouth_crop)

        # Run DeepFace analysis
        eye_result = DeepFace.analyze(img_path="eye_region.jpg", actions=['emotion'], enforce_detection=False)
        mouth_result = DeepFace.analyze(img_path="mouth_region.jpg", actions=['emotion'], enforce_detection=False)

        print("👀 Eye Region Emotions:", eye_result[0]['emotion'])
        print("👄 Mouth Region Emotions:", mouth_result[0]['emotion'])

else:
    print("No face detected.")


👀 Eye Region Emotions: {'angry': 2.430010193066323e-07, 'disgust': 3.8639415869941882e-19, 'fear': 0.0011287221241218504, 'happy': 0.0009147149285126943, 'sad': 2.7052702092356107e-09, 'surprise': 2.3790910647775165e-10, 'neutral': 99.99796152114868}
👄 Mouth Region Emotions: {'angry': 0.005284247065553092, 'disgust': 0.0018803059414999191, 'fear': 35.75203248882681, 'happy': 0.002038035988452478, 'sad': 51.42822282650743, 'surprise': 4.560845401032683e-06, 'neutral': 12.810538746182768}


I0000 00:00:1748246920.924378 2503399 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1748246920.926493 2504185 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748246920.933752 2504186 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748246920.937409 2504184 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


In [6]:
from datasets import load_dataset

ds = load_dataset("/Users/renan/code/reneiji/non_verbal_comm/notebooks/RA_note/confidence_dataset/FacialConfidence")

FileNotFoundError: Couldn't find any data file at /Users/renan/code/reneiji/non_verbal_comm/notebooks/RA_note/confidence_dataset/FacialConfidence.

## Trying out with masking

In [4]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

img_path = "/Users/renan/code/reneiji/non_verbal_comm/notebooks/RA_note/Genuine_Smile.jpg"
img = cv2.imread(img_path)
if img is None:
    print(f"⚠ Failed to load image. Check path: {img_path}")
    exit()
h, w, _ = img.shape
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = face_mesh.process(rgb_img)

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:

        # Define regions to mask
        eye_indices = [33, 133, 362, 263]   # outer eye corners
        mouth_indices = [61, 291, 0, 17]    # mouth corners, chin, nose

        def get_bbox(indices, pad_ratio=0.3):
            xs = [int(face_landmarks.landmark[i].x * w) for i in indices]
            ys = [int(face_landmarks.landmark[i].y * h) for i in indices]
            x_min, x_max = min(xs), max(xs)
            y_min, y_max = min(ys), max(ys)
            box_w = x_max - x_min
            box_h = y_max - y_min
            # Expand slightly
            x_min = max(0, int(x_min - box_w * pad_ratio))
            x_max = min(w, int(x_max + box_w * pad_ratio))
            y_min = max(0, int(y_min - box_h * pad_ratio))
            y_max = min(h, int(y_max + box_h * pad_ratio))
            return x_min, y_min, x_max, y_max

        eye_x1, eye_y1, eye_x2, eye_y2 = get_bbox(eye_indices)
        mouth_x1, mouth_y1, mouth_x2, mouth_y2 = get_bbox(mouth_indices)

        # Make copies of the original image
        mask_eye = img.copy()
        mask_mouth = img.copy()

        blur_mouth = cv2.GaussianBlur(mask_eye[mouth_y1:mouth_y2, mouth_x1:mouth_x2], (51, 51), 0)
        blur_eye = cv2.GaussianBlur(mask_mouth[0:eye_y2, :], (51, 51), 0)

        # Mask out mouth + chin for eye-focused image
        mask_eye[mouth_y1:mouth_y2, mouth_x1:mouth_x2] = blur_mouth

        # Mask out eyes + forehead for mouth-focused image
        mask_mouth[0:eye_y2, :] = blur_eye  # roughly top half

        # Optional: save or visualize masked images
        cv2.imwrite("masked_eye_focus.jpg", mask_eye)
        cv2.imwrite("masked_mouth_focus.jpg", mask_mouth)

        # Run DeepFace analysis
        eye_result = DeepFace.analyze(img_path="masked_eye_focus.jpg", actions=['emotion'], enforce_detection=False)
        mouth_result = DeepFace.analyze(img_path="masked_mouth_focus.jpg", actions=['emotion'], enforce_detection=False)

        print("👀 Eye-Focused Emotions:", eye_result[0]['emotion'])
        print("👄 Mouth-Focused Emotions:", mouth_result[0]['emotion'])

else:
    print("No face detected.")

👀 Eye-Focused Emotions: {'angry': 0.3137337975203991, 'disgust': 64.55342173576355, 'fear': 18.41873675584793, 'happy': 1.471702754497528, 'sad': 4.76096048951149, 'surprise': 1.0638452135026455, 'neutral': 9.417599439620972}
👄 Mouth-Focused Emotions: {'angry': 2.925314045428636e-14, 'disgust': 1.1485896445150053e-35, 'fear': 1.5336007138891326e-16, 'happy': 100.0, 'sad': 5.419644225589104e-11, 'surprise': 2.552502920138977e-06, 'neutral': 2.8781537157913928e-11}


I0000 00:00:1748310802.457964 2706939 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1748310802.460631 2764097 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748310802.469694 2764099 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


## Blurrying + video

In [2]:
import cv2
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

video_path = "/Users/renan/code/reneiji/non_verbal_comm/notebooks/RA_note/monologue_video.mp4"
cap = cv2.VideoCapture(video_path)

# Define output writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_blur.mp4', fourcc, int(cap.get(cv2.CAP_PROP_FPS)),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:

            def get_bbox(indices, pad_ratio=0.3):
                xs = [int(face_landmarks.landmark[i].x * w) for i in indices]
                ys = [int(face_landmarks.landmark[i].y * h) for i in indices]
                x_min, x_max = min(xs), max(xs)
                y_min, y_max = min(ys), max(ys)
                box_w = x_max - x_min
                box_h = y_max - y_min
                x_min = max(0, int(x_min - box_w * pad_ratio))
                x_max = min(w, int(x_max + box_w * pad_ratio))
                y_min = max(0, int(y_min - box_h * pad_ratio))
                y_max = min(h, int(y_max + box_h * pad_ratio))
                return x_min, y_min, x_max, y_max

            # Example: blur mouth
            mouth_indices = [61, 291, 0, 17]
            mx1, my1, mx2, my2 = get_bbox(mouth_indices)
            mouth_roi = frame[my1:my2, mx1:mx2]
            blurred_mouth = cv2.GaussianBlur(mouth_roi, (51, 51), 0)
            frame[my1:my2, mx1:mx2] = blurred_mouth

            # Example: blur eyes (optional)
            eye_indices = [33, 133, 362, 263]
            ex1, ey1, ex2, ey2 = get_bbox(eye_indices)
            eye_roi = frame[0:ey2, ex1:ex2]
            blurred_eye = cv2.GaussianBlur(eye_roi, (51, 51), 0)
            frame[0:ey2, ex1:ex2] = blurred_eye

    # Write frame to output
    out.write(frame)

    # Optional: show preview
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


I0000 00:00:1748308318.237882 2672173 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1748308318.239703 2704677 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748308318.247964 2704674 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


# Removing scenes where faces are not detected

In [ ]:
# Setup Mediapipe
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.6)

# Open webcam (use 0 for default camera, or provide video path)
cap = cv2.VideoCapture(0)  # replace 0 with "input_video.mp4" for file

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB for Mediapipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            # Get face bounding box
            bbox = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            x1 = int(bbox.xmin * w)
            y1 = int(bbox.ymin * h)
            x2 = x1 + int(bbox.width * w)
            y2 = y1 + int(bbox.height * h)

            # Ensure box is within frame
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)

            face_img = frame[y1:y2, x1:x2]

            try:
                # Analyze emotion with DeepFace
                result = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False)
                dominant_emotion = result[0]['dominant_emotion']

                # Draw bounding box and emotion label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, dominant_emotion, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            except Exception as e:
                print(f"DeepFace error: {e}")

    # Show live frame
    cv2.imshow('Live Emotion Analyzer', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1748392566.350694 3152210 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1748392566.364795 3426671 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


## Using emotion recognizer and heuristic approach to predict confidence

In [ ]:
import cv2
from deepface import DeepFace
import mediapipe as mp

# Setup Mediapipe
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.6)
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_frame)

    label = "No face"
    if results.detections:
        for detection in results.detections:
            # Get bounding box
            bbox = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            x1 = int(bbox.xmin * w)
            y1 = int(bbox.ymin * h)
            x2 = x1 + int(bbox.width * w)
            y2 = y1 + int(bbox.height * h)
            x1, y1, x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)

            face_img = frame[y1:y2, x1:x2]

            try:
                # Analyze emotion
                result = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False)
                emotion = result[0]['dominant_emotion']

                # Analyze landmarks
                mesh_results = face_mesh.process(rgb_frame)
                tension_score = 0
                if mesh_results.multi_face_landmarks:
                    landmarks = mesh_results.multi_face_landmarks[0].landmark

                    # Mouth tightness
                    top_lip = landmarks[13]
                    bottom_lip = landmarks[14]
                    lip_distance = abs(top_lip.y - bottom_lip.y)
                    if lip_distance < 0.02:
                        tension_score += 1  # tight mouth

                    # Eyebrow raise
                    left_brow = landmarks[65]
                    left_eye = landmarks[159]
                    brow_eye_gap = abs(left_brow.y - left_eye.y)
                    if brow_eye_gap > 0.05:
                        tension_score += 1  # raised brow

                    # Eye openness
                    left_upper = landmarks[159]
                    left_lower = landmarks[145]
                    eye_open = abs(left_upper.y - left_lower.y)
                    if eye_open > 0.04:
                        tension_score += 1  # wide eyes

                    # Jaw clench (side-to-side width check)
                    left_jaw = landmarks[234]
                    right_jaw = landmarks[454]
                    jaw_width = abs(left_jaw.x - right_jaw.x)
                    if jaw_width < 0.35:
                        tension_score += 1  # clenched jaw (narrower)

                # Heuristic rule
                if emotion in ['fear', 'surprise', 'disgust'] or tension_score >= 2:
                    label = "Nervous"
                elif emotion in ['happy', 'neutral'] and tension_score <= 1:
                    label = "Confident"
                else:
                    label = "Uncertain"

                # Draw box + label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} ({emotion}, tension={tension_score})", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

            except Exception as e:
                print(f"DeepFace error: {e}")

    # Show live video
    cv2.imshow('Confidence/Nervousness Estimator', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1748331090.563067 3149569 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1748331090.564689 3149917 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1748331090.568680 3149569 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1748331090.569678 3149929 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748331090.573011 3149928 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748331092.046464 3149932 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IM

# Using Pytorch for facial confidence estimation

In [2]:
# Step 1: Dataset wrapper
class FaceConfidenceDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img = self.df.iloc[idx]['image']
        label = self.df.iloc[idx]['label']

        # Ensure RGB
        if isinstance(img, str):
            img = Image.open(img).convert('RGB')
        else:
            img = img.convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img, torch.tensor(label, dtype=torch.long)

# Step 2: Transforms
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert to grayscale RGB
    transforms.Resize((224, 224)),                # Resize to match ResNet input
    transforms.ToTensor(),                        # Convert to tensor
    transforms.Normalize([0.485, 0.456, 0.406],   # Normalize (ImageNet mean)
                         [0.229, 0.224, 0.225])   # Normalize (ImageNet std)
])


In [ ]:
# Step 2.1: Get data set
ds = load_dataset("march18/FacialConfidence")
df_train = pd.DataFrame.from_dict(ds['train'])
df_test = pd.DataFrame.from_dict(ds['test'])

### Below is the first base model to predict confident face

In [ ]:
# Step 3: Dataloaders
train_dataset = FaceConfidenceDataset(df_train, transform)
test_dataset = FaceConfidenceDataset(df_test, transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Step 4: Model (pretrained ResNet18)
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes: confident, non-confident

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
model = model.to(device)

# Step 5: Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
epochs = 5

# Step 6: Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

# Step 7: Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/5, Loss: 0.4308
Epoch 2/5, Loss: 0.2816
Epoch 3/5, Loss: 0.1649
Epoch 4/5, Loss: 0.0888
Epoch 5/5, Loss: 0.0635
Test Accuracy: 0.8380


In [ ]:
# Saving the model
# torch.save(model.state_dict(), 'first_model.pth')

# Below is the code to load first_model.pth and make predictions

'''
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 2)  # same output setup

model.load_state_dict(torch.load('first_model.pth'))
model.to(device)
'''

"\nmodel = models.resnet18(pretrained=False)\nmodel.fc = nn.Linear(model.fc.in_features, 2)  # same output setup\n\nmodel.load_state_dict(torch.load('first_model.pth'))\nmodel.to(device)\n"

### Second model training with higher epochs

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Datasets & Dataloaders
batch_size = 64

train_dataset = FaceConfidenceDataset(df_train, transform)
test_dataset = FaceConfidenceDataset(df_test, transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)

# Loss, Optimizer, Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)

epochs = 15
best_val_acc = 0

for epoch in range(epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    train_preds, train_labels = [], []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(train_labels, train_preds)
    avg_train_loss = running_loss / len(train_loader)

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_acc = accuracy_score(val_labels, val_preds)

    # Step the scheduler
    scheduler.step(avg_val_loss)

    print(f"Epoch {epoch + 1}/{epochs} "
          f"Train Loss: {avg_train_loss:.4f} "
          f"Train Acc: {train_acc:.4f} "
          f"Val Loss: {avg_val_loss:.4f} "
          f"Val Acc: {val_acc:.4f}")

    # Save best model (optional)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"✅ Best model saved at epoch {epoch + 1} with val acc: {val_acc:.4f}")


Epoch 1/15 Train Loss: 0.4328 Train Acc: 0.7927 Val Loss: 0.3685 Val Acc: 0.8373
✅ Best model saved at epoch 1 with val acc: 0.8373
Epoch 2/15 Train Loss: 0.2816 Train Acc: 0.8811 Val Loss: 0.3915 Val Acc: 0.8267
Epoch 3/15 Train Loss: 0.1636 Train Acc: 0.9351 Val Loss: 0.5023 Val Acc: 0.8292
Epoch 4/15 Train Loss: 0.0852 Train Acc: 0.9686 Val Loss: 0.5520 Val Acc: 0.8321
Epoch 5/15 Train Loss: 0.0265 Train Acc: 0.9919 Val Loss: 0.5519 Val Acc: 0.8508
✅ Best model saved at epoch 5 with val acc: 0.8508
Epoch 6/15 Train Loss: 0.0095 Train Acc: 0.9981 Val Loss: 0.6337 Val Acc: 0.8536
✅ Best model saved at epoch 6 with val acc: 0.8536
Epoch 7/15 Train Loss: 0.0054 Train Acc: 0.9989 Val Loss: 0.6768 Val Acc: 0.8465
Epoch 8/15 Train Loss: 0.0033 Train Acc: 0.9994 Val Loss: 0.6537 Val Acc: 0.8532
Epoch 9/15 Train Loss: 0.0019 Train Acc: 0.9996 Val Loss: 0.6775 Val Acc: 0.8537
✅ Best model saved at epoch 9 with val acc: 0.8537
Epoch 10/15 Train Loss: 0.0015 Train Acc: 0.9996 Val Loss: 0.6974 V

## Loading the best model for detecting facial confidence

In [4]:
# Recreate the model architecture
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: confident (0), non-confident (1)

# Load saved weights
model.load_state_dict(torch.load('best_model.pth', map_location='cpu'))  # change to 'cuda' if using GPU
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Plotting confusion matrix and classification report for the best model

In [ ]:
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)

# Plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Confident', 'Non-Confident'])
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix on Test Set')
plt.show()

In [17]:
report = classification_report(all_labels, all_preds, target_names=['Confident', 'Non-Confident'])

print("Classification Report:")
print(report)

Classification Report:
               precision    recall  f1-score   support

    Confident       0.87      0.78      0.82      3007
Non-Confident       0.85      0.91      0.88      4171

     accuracy                           0.86      7178
    macro avg       0.86      0.85      0.85      7178
 weighted avg       0.86      0.86      0.85      7178



## Simulating a real-time webcam feed with the trained model

In [ ]:
# Setup Mediapipe
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.load_state_dict(torch.load('best_model.pth', map_location=device))
model.to(device)
model.eval()

mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.6)

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB for Mediapipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            h, w, _ = frame.shape
            x1 = int(bbox.xmin * w)
            y1 = int(bbox.ymin * h)
            x2 = x1 + int(bbox.width * w)
            y2 = y1 + int(bbox.height * h)

            # Ensure box is within frame
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)

            face_img = frame[y1:y2, x1:x2]

            try:
                # Analyze emotion with DeepFace
                result = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False)
                dominant_emotion = result[0]['dominant_emotion']

                # Predict confidence with your model
                pil_face = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))
                input_tensor = transform(pil_face).unsqueeze(0).to(device)
                output = model(input_tensor)
                _, pred = torch.max(output, 1)
                confidence_label = 'Confident' if pred.item() == 0 else 'Non-Confident'

                # Combine labels
                combined_label = f"{dominant_emotion}, {confidence_label}"

                # Draw bounding box and labels
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, combined_label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            except Exception as e:
                print(f"Error: {e}")

    cv2.imshow('Live Emotion + Confidence Analyzer', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1748394111.316820 3455154 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1748394111.318373 3465987 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' is not defined
Error: name 'model' 